In [45]:
import vk_api
from vk_api.exceptions import ApiError
import pandas as pd
from math import ceil
from datetime import datetime
from tqdm import tqdm
import re

In [4]:
ACCESS_TOKEN = 'ACCESS_TOKEN'
GROUP_ID = '9online'
DATA_PER_REQUEST = 1000

In [5]:
vk_session = vk_api.VkApi(token=ACCESS_TOKEN)

vk = vk_session.get_api()

# Get list of group members

In [6]:
# Check connection
response = vk.groups.getById(group_id = GROUP_ID, fields='members_count')

In [7]:
response

[{'id': 60463565,
  'members_count': 76398,
  'name': 'Девяткино online | Мурино',
  'screen_name': '9online',
  'is_closed': 0,
  'type': 'page',
  'photo_50': 'https://sun9-58.userapi.com/s/v1/if1/4PPz8WVpAvhvXWd3fFx4fFjEdKdeOVcBQePRYyAZf0QfsY7eQOQoxPBOuRJuptoX66KfpZwc.jpg?size=50x50&quality=96&crop=48,618,1118,1118&ava=1',
  'photo_100': 'https://sun9-58.userapi.com/s/v1/if1/VGE9nHDm65ViAZYNFtWHI0Dd0GJcUxpBmsu1TJiEk2k1bV5aUOMtwlnOKWQU0hd3voRluZJF.jpg?size=100x100&quality=96&crop=48,618,1118,1118&ava=1',
  'photo_200': 'https://sun9-58.userapi.com/s/v1/if1/8Hh7oZ43xNRm8-hzm5HK3C_8IdrvV3PujX6rA0nsNwJN7Rr5KLebegExTxUVTtLSerEsl_Gz.jpg?size=200x200&quality=96&crop=48,618,1118,1118&ava=1'}]

In [9]:
# Get members list from group
members_id_list = []
print(f'{datetime.now()} Start copying data from vk.com/{GROUP_ID}')
uni_data = vk.groups.getById(group_id=GROUP_ID, fields='members_count')
members_count = uni_data[0]['members_count']
for i in range(ceil(members_count / DATA_PER_REQUEST)):
    members_pesponse = vk.groups.getMembers(group_id=GROUP_ID, count=DATA_PER_REQUEST, offset=i*DATA_PER_REQUEST, sort='id_asc')
    members_id_list.extend(members_pesponse['items'])
print(f'{datetime.now()} Done copying data from vk.com/{GROUP_ID}. Total users count: {members_count}')

2023-12-17 19:11:17.562026 Start copying data from vk.com/9online
2023-12-17 19:11:49.615337 Done copying data from vk.com/9online. Total users count: 76398


In [16]:
len(members_id_unique)

76388

# Get connections between people

In [19]:
def get_friends_list(api, id):
  data = api.friends.get(user_id=id, sort='id_asc')
  friends_count = data['count']
  friends = data['items']
  if(friends_count > 5000):
    for i in range(ceil(friends_count / 5000)):
      data = api.friends.get(user_id=id, offset=i*5000, sort='id_asc')
      friends.extend(data['items'])
  return friends

In [20]:
# Get friends list of the all members and users data
private_profiles = []
friendship_data = {} # Dict ('user_id': [user_ids]) described list of key user's friends
print(f'{datetime.now()} Start requesting data')
for id in tqdm(members_id_unique):
  try:
    friends_list = get_friends_list(vk, id)
    if id not in friendship_data:
      friendship_data[id] = friends_list
  except ApiError:
    private_profiles.append(id)

print(f'{datetime.now()} End requesting data')
print(f'There are {len(private_profiles)} private profiles among users')

2023-12-17 19:14:59.361938 Start requesting data


100%|██████████| 76388/76388 [9:14:39<00:00,  2.30it/s]   

2023-12-18 04:29:38.787547 End requesting data
There are 29172 private profiles among users


In [28]:
connections = []
for key in tqdm(friendship_data):
  for friend in friendship_data[key]:
    buff = [key, friend] if key < friend else [friend, key]
    if friend in members_id_unique and friend not in private_profiles:
        connections.append(buff)

connections[0]

100%|██████████| 47216/47216 [5:58:59<00:00,  2.19it/s]   


[502770170, 515899396]

In [29]:
friends_edges = pd.DataFrame(connections, columns=['id1','id2']).drop_duplicates()
friends_edges.head(10)

,id1,id2
0,502770170,515899396
1,515024636,515899396
2,515898779,515899396
3,515899396,516794208
4,515899396,565177928
5,19833,20840460
6,1242248,20840460
7,1536882,20840460
8,1895899,20840460
9,2482685,20840460


In [30]:
friends_edges.shape[0]

169920

In [31]:
friends_edges.to_csv('edges.csv', index=False)

# Get users data

In [21]:
print(f'{datetime.now()} Start requesting profile\'s data')
users_data_list = []
for user in tqdm(members_id_unique):
    if user not in (private_profiles):
        res = vk.users.get(user_ids=user, fields=['career', 'city', 'country', 'home_town', 'education', 'sex', 'occupation', 'relation'])
        users_data_list.append(res)
print(f'{datetime.now()} End requesting profile\'s data')

2023-12-18 04:29:38.793187 Start requesting profile's data


100%|██████████| 76388/76388 [5:27:30<00:00,  3.89it/s]   

2023-12-18 09:57:09.527489 End requesting profile's data


In [25]:
res = vk.users.get(user_ids=members_id_unique[0], fields=['city', 'country', 'home_town', 'education', 'sex', 'occupation', 'relation'])
res

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [71]:
users_data = []
for user in users_data_list:
  users_data.append([
      user[0]['id'] if 'id' in user[0] else None,
      user[0]['city']['id'] if 'city' in user[0] else None,
      user[0]['city']['title'].replace('\r\n', '').replace('\n', '') if 'city' in user[0] else None,
      user[0]['country']['id'] if 'country' in user[0] else None,
      user[0]['country']['title'].replace('\r\n', '') if 'country' in user[0] else None,
      user[0]['home_town'].replace('\r\n', '').replace('\n', '') if 'home_town' in user[0] else None,
      user[0]['university'] if 'university' in user[0] else None,
      user[0]['university_name'].replace('\r\n', '') if 'university_name' in user[0] else None,
      user[0]['faculty'] if 'faculty' in user[0] else None,
      user[0]['faculty_name'].replace('\r\n', '') if 'faculty_name' in user[0] else None,
      user[0]['graduation'] if 'graduation' in user[0] else None,
      user[0]['sex'] if 'sex' in user[0] else None,
      user[0]['occupation']['type'] if 'occupation' in user[0] and 'type' in user[0]['occupation'] else None,
      user[0]['occupation']['id'] if 'occupation' in user[0] and 'id' in user[0]['occupation'] else None,
      user[0]['occupation']['name'].replace('\r\n', '') if 'occupation' in user[0] and 'name' in user[0]['occupation'] else None,
      user[0]['relation'] if 'relation' in user[0] else None
  ])

In [72]:
nodes = pd.DataFrame(users_data, 
                     columns=['id','city_id', 'city_title', 'country_id', 'country_title',
                             'home_town', 'university', 'university_name', 'faculty', 'faculty_name',
                             'graduation', 'sex', 'occupation_type', 'occupation_id',
                             'occupation_name', 'relation']
                    ).drop_duplicates()

nodes.head(10)

,id,city_id,city_title,country_id,country_title,home_town,university,university_name,faculty,faculty_name,graduation,sex,occupation_type,occupation_id,occupation_name,relation
0,515899396,2.0,Saint Petersburg,1.0,Russia,,0.0,,0.0,,0.0,1,work,161114502.0,"""Магнит"". Строим карьеру и развиваемся вместе!",0.0
1,20840460,2.0,Saint Petersburg,1.0,Russia,None,NaN,None,NaN,None,NaN,2,None,NaN,None,NaN
2,39059473,2.0,Saint Petersburg,1.0,Russia,None,NaN,None,NaN,None,NaN,1,university,650.0,НГЛУ им. Н. А. Добролюбова,NaN
3,186122258,NaN,None,NaN,None,None,NaN,None,NaN,None,NaN,1,None,NaN,None,NaN
4,415236121,NaN,None,NaN,None,None,NaN,None,NaN,None,NaN,2,university,685.0,ЮРГПУ (НПИ) им. М. И. Платова (бывш. ЮРГТУ (НПИ)),NaN
5,244580379,2.0,Saint Petersburg,1.0,Russia,None,NaN,None,NaN,None,NaN,1,work,NaN,https://vk.com/angieyoga,NaN
6,7995421,2.0,Saint Petersburg,1.0,Russia,Нукус,0.0,,0.0,,0.0,1,None,NaN,None,6.0
7,217972765,2.0,Saint Petersburg,1.0,Russia,None,NaN,None,NaN,None,NaN,2,university,7860.0,ВТИ (СПВВТКУ),NaN
8,309067806,NaN,None,NaN,None,None,NaN,None,NaN,None,NaN,2,None,NaN,None,NaN
9,84148257,2.0,Saint Petersburg,1.0,Russia,None,NaN,None,NaN,None,NaN,1,university,555.0,"ВятГУ (бывш. ВятГТУ, КирПИ)",NaN


In [75]:
nodes.to_csv('nodes.csv', index=False)

In [74]:
len(users_data)

47216

In [73]:
nodes.loc[nodes["id"]==804765199]

,id,city_id,city_title,country_id,country_title,home_town,university,university_name,faculty,faculty_name,graduation,sex,occupation_type,occupation_id,occupation_name,relation
41171,804765199,1120143.0,Pochinok (glebovskaya S/a P/o Glebovo),1.0,Russia,Севастополь,0.0,,0.0,,0.0,1,None,NaN,None,3.0
